# Evaluation (CPU-friendly)

This notebook demonstrates CPU-only evaluation of the fragment autoencoder using a subsample of the validation dataset.

Requirements:
- Data present under `data/dev_data` (or update the path below).
- Git LFS installed and pulled so checkpoints/data are available.


In [ ]:
%env CUDA_VISIBLE_DEVICES=
import os
from pathlib import Path
import torch
import pytorch_lightning as pl

from src.study.trainer import FragmentAutoencoderTrainer

# User-configurable paths
DATA_PATH = Path('data/dev_data')  # or 'data/train_data'
CHECKPOINT_DIR = Path('outputs/conv_autoencoder/checkpoints')

# Find the best checkpoint produced by ModelCheckpoint callback
ckpts = sorted(CHECKPOINT_DIR.glob('best_model-*.ckpt'))
assert ckpts, f'No checkpoint found under {CHECKPOINT_DIR}. Run training first or update path.'
best_ckpt = ckpts[0]
best_ckpt

In [ ]:
# Load model on CPU
model = FragmentAutoencoderTrainer.load_from_checkpoint(
    best_ckpt,
    map_location=torch.device('cpu'),
 )

# Point the trainer to the evaluation data and set up datasets
model.data_path = DATA_PATH
model.setup(stage='fit')  # creates train/val datasets; we will use val_dataloader

trainer = pl.Trainer(
    accelerator='cpu',
    devices=1,
    logger=False,
)
val_results = trainer.validate(model)
val_results

## Notes
- Metrics include ARI/NMI (KMeans and KMedoids) and aggregated means logged by the Lightning module.
- For a quick smoke test without training, you can point to any available checkpoint in `outputs/.../checkpoints/`.
